# Template for CLEM segmentation

This template contains the basic steps for CLEM analysis.
1. Stitch EM image(s)
2. Invert EM image(s)
3. Register (correlate) EM and LM images
4. Define Cell Mask
5. Segment EM images
    5.1 Automatically segment EM images using AI (Mask-RCNN)
    5.2 Manually correct (or create from scratch) segmentation
6. Get statistics

## 1. Stitch EM 
EM images are aquired in multiple tiles using SerialEM (in the Taraska-Lab at NHLBI, NIH).
The first step of image analysis is stitching of the tiles to form a single image. This is done in IMOD using sloppyblend.

**--> A custom-written bash script by Dr Sochacki can be pulled from local gitlab repository linux_batch_EMstitching.**

Notes:
 - Must have IMOD installed
  - Use IMOD version 4.11 or higher!
    In older versions, the pixel spacing is saved in dots/inch instead of dots/centimeter which creates problems when reading pixel size from metadata in the rest of the analysis pipeline.
  - If you have an older IMOD version installed, you can circumvent this problem by manually updating the pixel size to µm units in FIJI.
    Open image in FIJI > Analyze > Set Scale... > set the correct pixel size and unit > OK

## 2. Invert EM-image(s)
Historically, we have used inverted PREM images for CLEM analysis.
The ai-based segmentation is trained on inverted images and most of the code in the pyclem package expects inverted images.

For further analysis, file names for EM-images should end with "_inv.tif"

- You can invert images using FIJI:
    1. Open image in FIJI
    2. Edit > Invert
    3. Save image

- **Or use the python script for batch-processing included in pyclem (see below)**
    </br>
    The code uses regular expression patterns to find all files in a directory matching a certain pattern.
    --> To (VERY briefly) learn about regular expressions, see: https://www.dataquest.io/blog/regex-cheatsheet/

In [ ]:
from pathlib import Path
from pyclem.file_io import get_files
from pyclem.scripts import invert_and_save

# Define path to directory with EM images
parent_directory = r'PATH_to_EM_images'
# Define regular expression pattern to match the filenames of YOUR EM-images
filename_pattern = r'.*cell.*\.tif$'

# Retrieve names of all files in the directory matching the pattern
filenames, _ = get_files(subdir=parent_directory, pattern=filename_pattern)
# Add parent directory to filenames to get full paths
filenames = [Path(parent_directory, filename) for filename in filenames]
# Invert images
invert_and_save(fn=filenames)
print('Done!')

## 3. Register (correlate) EM and LM images
For image registration, I recommend using the plugin "affinder" for the open-source python image analysis software "Napari".

Until the plugin is available in the official napari plugin repository, it can be installed from the local gitlab repository "affinder".
To do so, follow these steps:
 - Open conda prompt
 - activate your pyclem environment
 - Install the plugin directly from github with:
   - **python -m pip install git+https://github.com/jni/affinder.git**
 - Only do this once! The plugin will be available in napari from now on.

To use the plugin:
1. Open a napari viewer (see code below)
2. Open images to register (eg. EM, TIRF and STORM image) using the aicsimageio plugin. This ensures that the images are opened with the correct metadata (most importantly the pixel size).
    File > Open with plugin > Open File(s) > Open > select napari-aicsimageio as reader, when prompted
3. Open the affinder plugin
    Plugins > affinder > Start affinder
4. Select EM image as reference and LM image as moving
5. (Optional) Select location to save the transformation matrix
6. Click **"Start"** to start the manual registration process
7. Select three points in the reference image
8. Select the corresponding three points in the moving image (**in the same order!**)
    Affinder will perform a first calculation of the transformation matrix and apply it to the moving image
9. From here, you can iteratively add single points to improve the registration
10. When you are satisfied, click **"Finish"** to save the transformation matrix and exit the process.

To copy transformation from one image to another (e.g. from TIRF to STORM):
1. Open affinder widget to copy transformation
    Plugins > affinder > Copy affine
2. Set the already transformed image (e.g. TIRF) as src_layer and the untransformed image (e.g. STORM) as dst_layer
3. Click **"Copy"** to apply the transformation to the dst_layer
4. You can now use the "Start affinder" widget to further improve the registration, using the transformed image (e.g. STORM) as moving and the EM-image as reference.

To apply the transformation to an image (creating a new image):
1. Open affinder widget to apply transformation
    Plugins > affinder > Apply affine
2. Select the EM-image as reference layer and the transformed LM-image as moving layer
3. Click **"Apply"** to create a new image with the transformation applied.
4. You now use the built in napari functionality to save the new image.
    File > Save Selected Layer(s)... > Save (use napari builtins lossless)

To load previously saved transformation matrix:
1. Open affinder widget to load transformation
    Plugins > affinder > Load affine
2. Select the image you want to transform (e.g. LM-image) as layer
3. Select File to set the path to a saved transformation matrix
4. Click **"Load"** to apply the transformation to the selected layer

In [ ]:
import napari

viewer = napari.Viewer()


## 4. Define Cell Mask
Use the custom-written Napari widget cellmask_widget to create and save a binary cell mask.
1. Open a napari viewer and add the widget (see code below)
2. Select an (inverted) EM image from the widget.
3. Click **"Start"**
   - The image will be opened together with an empty shapes layer called "Cell Mask"
   - When opening images, cellmask_widget will automatically rescale the image to a pixelsiize of x nm/pixel as defined by the slider.
     This is to improve performance and reduce memory usage.
4. Use Napari shapes functionalities to cover areas you want to **exlude** from analysis.
5. When done, click **"Finish and Save"** to translate the shapes into a binary save it as a tif-file with the addendum "..._cellmask.tif".

In [ ]:
from pyclem.cellmask import cellmask_main

cellmask_main()

## 5.1 Automatically segment EM images using AI (Mask-RCNN)
The segmentation of EM images is done using a Mask-RCNN model trained on manually segmented EM-images.
The model is implemented in a custom-written python package called prem-segmentation.
**--> see local gitlab repository PREM-seg**

To this end, (inverted!) EM images are scaled to a pixelsize of 2.46 nm/px and split into overlapping tiles. Each tiles is then segmented utilizing the open source deep learning algorithm "Mask R-CNN". Segmented tiles are then scaled back to the original pixel size and stitched to a full image mask.

**NOTE: Make sure to run this next cell in the appropriate environment (mrcnn-env).**

In [ ]:
from prem_seg.main import main

main()

## 5.2 Manually correct (or create from scratch) segmentation
The automatic segmentation is not perfect and often needs manual correction.
To this end, use the Napari widget "checkseg" included in the pyclem package.

1. Open a napari viewer and add the widget (see code below)
2. Select an (inverted) EM-image
   - The paramter tile size defines the size of image tiles in µm. Default is 3x3 µm.
3. Click **"Start"**
   - checkseg_widget will automatically open the corresponding segmentation mask (if it exists in the same folder)
        - The segmentation mask will be translated into three shapes layers: "Domes", "Flats", "Spheres"
   - checkseg_widget will automatically consider the cellmask (if it exists in the same folder)
   - checkseg_widget will automatically progress to the first tile containing relevant data (i.e. not excluded by the cellmask, if available)
4. Use Napari shapes functionalities to correct segmentation results in current tile
5. When done, click **"Next"** to progress to the next tile
6. When done with all tiles, checkseg_widget will show the whole cell without tiles for a final check.
7. When done, click **"Finish and Save"** to translate the shapes into an RGB-mask and save it as a tif-file with the addendum "..._segmask_check.tif".


In [ ]:
from pyclem.checkseg import checkseg_main

checkseg_main()

## 6. Get statistics